In [17]:
import xml.etree.cElementTree as ET
import pprint
from collections import defaultdict
import re

raw_xml = "denver_colorado_osm.xml"

#mytree = ET.parse(raw_xml)
#myroot = mytree.getroot()

#for x in myroot.findall('way'):
 #   id = x.find('id').attrib
  #  print(id)
    
    
#The function below calculates all of the types of tags that we are dealing
#with in our XML file, and counts how many of each type we are dealing with
def count_all_tags(xmlfile):
    
    tag_counts = {}
    for tag in ET.iterparse(xmlfile):
        current = tag[1].tag
        tag_counts[current] = tag_counts.get(current, 0) + 1
        tag_counts = defaultdict(int)
    for tag in ET.iterparse(xmlfile):
        current = tag[1].tag
        tag_counts[current] += 1
        
    return tag_counts

count_all_tags(raw_xml)


defaultdict(int,
            {'note': 1,
             'meta': 1,
             'bounds': 1,
             'node': 3132214,
             'tag': 1842489,
             'nd': 3526619,
             'way': 396041,
             'member': 44664,
             'relation': 1725,
             'osm': 1})

In [18]:
#"lower", for tags that contain only lowercase letters and are valid,
#"lower_colon", for otherwise valid tags with a colon in their names,
#"problemchars", for tags with problematic characters, and
#"other", for other tags that do not fall into the other three categories.

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

def key_type(element, keys):
    
    if element.tag == "tag":
        k_value = element.attrib['k']
        if lower.search(k_value) is not None:
            keys['lower'] += 1
        elif lower_colon.search(k_value) is not None:
            keys['lower_colon'] += 1
        elif problemchars.search(k_value) is not None:
            keys["problemchars"] += 1
        else:
            keys['other'] += 1
            
    return keys

def process_map(filename):
    
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

process_map(raw_xml)

{'lower': 1292136, 'lower_colon': 531566, 'problemchars': 1, 'other': 18786}

In [21]:
#This function returns a list of all unique contributors

def unique_users(filename):
    
    users = set()
    for not_used, element in ET.iterparse(filename):
        if element.tag == "node" or element.tag == "way" or element.tag == "relation":
            users.add(element.attrib['uid'])

    return users
    
unique_users(raw_xml)


{'6186031',
 '3069035',
 '454093',
 '7157463',
 '3519193',
 '3250133',
 '1647868',
 '2098497',
 '6460266',
 '1817244',
 '1339694',
 '5667757',
 '6799903',
 '4097453',
 '3762268',
 '27541',
 '104962',
 '1237170',
 '385027',
 '2554698',
 '5540115',
 '304169',
 '8072170',
 '5475901',
 '12982',
 '3346538',
 '61942',
 '431126',
 '4211948',
 '554973',
 '693098',
 '883016',
 '1795336',
 '6794493',
 '1931857',
 '334722',
 '497227',
 '1721969',
 '29320',
 '28378',
 '1946151',
 '5027347',
 '977450',
 '236172',
 '736220',
 '8144806',
 '3675007',
 '1722865',
 '2547262',
 '69853',
 '6659373',
 '2103318',
 '5631771',
 '3207726',
 '362431',
 '363969',
 '2226712',
 '70696',
 '6217684',
 '5492566',
 '37392',
 '5201379',
 '8143935',
 '6240515',
 '5428681',
 '2193331',
 '3406980',
 '1442206',
 '5689995',
 '255976',
 '36223',
 '30554',
 '139546',
 '6873504',
 '396743',
 '2729990',
 '231017',
 '152294',
 '115468',
 '5974',
 '350283',
 '6755713',
 '4276714',
 '604586',
 '7787709',
 '414016',
 '3445434',
 '2

In [22]:
def count_unique_users():
    
    users = unique_users(raw_xml)
    print (len(users))
    
count_unique_users()

1476


In [32]:
#!/usr/bin/env python

# -*- coding: utf-8 -*-


import xml.etree.ElementTree as ET  # Use cElementTree or lxml if too slow

 

OSM_FILE = raw_xml  # Replace this with your osm file

SAMPLE_FILE = "sampledenver.xml"


k = 280  # Parameter: take every k-th top level element

def get_element(OSM_FILE, tags=('node', 'way', 'relation')):

   

    context = iter(ET.iterparse(OSM_FILE, events=('start', 'end')))

    _, root = next(context)

    for event, elem in context:

        if event == 'end' and elem.tag in tags:

            yield elem

            root.clear()

 

with open(SAMPLE_FILE, 'wb') as output:

#   ET.tostring(element, encoding='utf-8')

   

    # Write every kth top level element

    for i, element in enumerate(get_element(OSM_FILE)):

        if i % k == 0:

            output.write(ET.tostring(element, encoding='utf-8'))



In [1]:
#This is my main program for my submission
#!/usr/bin/env python
# -*- coding: utf-8 -*-

"""
After auditing is complete the next step is to prepare the data to be inserted into a SQL database.
To do so you will parse the elements in the OSM XML file, transforming them from document format to
tabular format, thus making it possible to write to .csv files.  These csv files can then easily be
imported to a SQL database as tables.

The process for this transformation is as follows:
- Use iterparse to iteratively step through each top level element in the XML
- Shape each element into several data structures using a custom function
- Utilize a schema and validation library to ensure the transformed data is in the correct format
- Write each data structure to the appropriate .csv files

We've already provided the code needed to load the data, perform iterative parsing and write the
output to csv files. Your task is to complete the shape_element function that will transform each
element into the correct format. To make this process easier we've already defined a schema (see
the schema.py file in the last code tab) for the .csv files and the eventual tables. Using the 
cerberus library we can validate the output against this schema to ensure it is correct.

## Shape Element Function
The function should take as input an iterparse Element object and return a dictionary.

### If the element top level tag is "node":
The dictionary returned should have the format {"node": .., "node_tags": ...}

The "node" field should hold a dictionary of the following top level node attributes:
- id
- user
- uid
- version
- lat
- lon
- timestamp
- changeset
All other attributes can be ignored

The "node_tags" field should hold a list of dictionaries, one per secondary tag. Secondary tags are
child tags of node which have the tag name/type: "tag". Each dictionary should have the following
fields from the secondary tag attributes:
- id: the top level node id attribute value
- key: the full tag "k" attribute value if no colon is present or the characters after the colon if one is.
- value: the tag "v" attribute value
- type: either the characters before the colon in the tag "k" value or "regular" if a colon
        is not present.

Additionally,

- if the tag "k" value contains problematic characters, the tag should be ignored
- if the tag "k" value contains a ":" the characters before the ":" should be set as the tag type
  and characters after the ":" should be set as the tag key
- if there are additional ":" in the "k" value they and they should be ignored and kept as part of
  the tag key. For example:

  <tag k="addr:street:name" v="Lincoln"/>
  should be turned into
  {'id': 12345, 'key': 'street:name', 'value': 'Lincoln', 'type': 'addr'}

- If a node has no secondary tags then the "node_tags" field should just contain an empty list.

The final return value for a "node" element should look something like:

{'node': {'id': 757860928,
          'user': 'uboot',
          'uid': 26299,
       'version': '2',
          'lat': 41.9747374,
          'lon': -87.6920102,
          'timestamp': '2010-07-22T16:16:51Z',
      'changeset': 5288876},
 'node_tags': [{'id': 757860928,
                'key': 'amenity',
                'value': 'fast_food',
                'type': 'regular'},
               {'id': 757860928,
                'key': 'cuisine',
                'value': 'sausage',
                'type': 'regular'},
               {'id': 757860928,
                'key': 'name',
                'value': "Shelly's Tasty Freeze",
                'type': 'regular'}]}

### If the element top level tag is "way":
The dictionary should have the format {"way": ..., "way_tags": ..., "way_nodes": ...}

The "way" field should hold a dictionary of the following top level way attributes:
- id
-  user
- uid
- version
- timestamp
- changeset

All other attributes can be ignored

The "way_tags" field should again hold a list of dictionaries, following the exact same rules as
for "node_tags".

Additionally, the dictionary should have a field "way_nodes". "way_nodes" should hold a list of
dictionaries, one for each nd child tag.  Each dictionary should have the fields:
- id: the top level element (way) id
- node_id: the ref attribute value of the nd tag
- position: the index starting at 0 of the nd tag i.e. what order the nd tag appears within
            the way element

The final return value for a "way" element should look something like:

{'way': {'id': 209809850,
         'user': 'chicago-buildings',
         'uid': 674454,
         'version': '1',
         'timestamp': '2013-03-13T15:58:04Z',
         'changeset': 15353317},
 'way_nodes': [{'id': 209809850, 'node_id': 2199822281, 'position': 0},
               {'id': 209809850, 'node_id': 2199822390, 'position': 1},
               {'id': 209809850, 'node_id': 2199822392, 'position': 2},
               {'id': 209809850, 'node_id': 2199822369, 'position': 3},
               {'id': 209809850, 'node_id': 2199822370, 'position': 4},
               {'id': 209809850, 'node_id': 2199822284, 'position': 5},
               {'id': 209809850, 'node_id': 2199822281, 'position': 6}],
 'way_tags': [{'id': 209809850,
               'key': 'housenumber',
               'type': 'addr',
               'value': '1412'},
              {'id': 209809850,
               'key': 'street',
               'type': 'addr',
               'value': 'West Lexington St.'},
              {'id': 209809850,
               'key': 'street:name',
               'type': 'addr',
               'value': 'Lexington'},
              {'id': '209809850',
               'key': 'street:prefix',
               'type': 'addr',
               'value': 'West'},
              {'id': 209809850,
               'key': 'street:type',
               'type': 'addr',
               'value': 'Street'},
              {'id': 209809850,
               'key': 'building',
               'type': 'regular',
               'value': 'yes'},
              {'id': 209809850,
               'key': 'levels',
               'type': 'building',
               'value': '1'},
              {'id': 209809850,
               'key': 'building_id',
               'type': 'chicago',
               'value': '366409'}]}
"""

import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET
import schema
import six
from collections import defaultdict

OSM_PATH = "denver_colorado_osm.xml"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
street_types = defaultdict(set)

# Expected Values for Street Name
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place",
           "Square", "Lane", "Road", "Parkway", "Commons", "Way", "Circle"]

# Incorrect Street Type Values to change
mapping = {"Blvd": "Boulevard",
           "Ave": "Avenue",
           "St": "Street",
           "Pl": "Place"}

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)
            
#Auditing OSM_PATH file
def audit(OSM_PATH):
    osm_file = OSM_PATH
    street_types = defaultdict(set)
    
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    return street_types

#Takes name and maps as parameters, updates name and returns new name
def update_name(name, maps):
    target = ''
    for key in maps.keys():
        if key in name:
            target = key
    name = name.replace(target, maps[target])
    return name

#Main function to shape elements for conversion to CSV
def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    if element.tag == 'node':
        
        for item in node_attr_fields:
             node_attribs[item] = element.get(item)
            
        for child in element:
            tag_store = {}
            find_colon = child.get('k').find(':')
            
            if (child.tag == 'tag'):
                tag_store['id'] = element.get('id')
                tag_store['value'] = child.get('v')
                
                if (find_colon != -1):
                    type_value = child.get('k')[:find_colon]
                    key_value = child.get('k')[find_colon+1:]
                    tag_store['type'] = type_value
                    tag_store['key'] = key_value
                    
                else:
                    tag_store['key'] = child.get('k')
                    tag_store['type'] = 'regular'
                    
                tags.append(tag_store)
                
        return {'node': node_attribs, 'node_tags': tags}
    
    elif element.tag == 'way':
        
        for item in way_attr_fields:
            n = 0
            way_attribs[item] = element.get(item)
        
        for child in element:
            
            if child.tag == 'nd':
                nd_dict = {}
                nd_dict['id'] = element.get('id')
                nd_dict['node_id'] = child.get('ref')
                nd_dict['position'] = n
                n += 1
                way_nodes.append(nd_dict)
                
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.items())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, str) else v) for k, v in row.items()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file,          codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file,          codecs.open(WAYS_PATH, 'w') as ways_file,          codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file,          codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        #validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.

    process_map(OSM_PATH, validate=False)
   
    

